# Mutations in genetic algorithms

**Mutation** is the process of randomly altering some symbols in a solution's string. Mutation can be performed with different probabilities, such as fixed, adaptive, etc. <span style="color:#009988">The idea is to introduce some diversity and variation in the population, which may help to escape from local optima and explore new regions of the search space.</span>

Large initial mutation rate (probability that mutation occur in an individual) favors exploration due to causing large jumps in search space. As mutation rate decreases, exploitation is favored. <span style="color:#998800">The probability of mutation could be proportional to the fitness of the individual, or we can simply decrease the mutation probability over time.</span>

<figure>
    <center> <img src="./pics/Mutation/mutation.png"  alt='missing' width="400"  ><center/>
<figure/>

## **Contents**

<a href='#chrom-def'>01. Chromosome Definition</a>  
 
<a href='#bit-flip'>02. Bit Flip Mutation</a>

<a href='#random-resetting'>03. Random Resetting Mutation</a>

<a href='#swap'>04. Swap Mutation</a>

<a href='#scramble'>05. Scramble Mutation or Partial Shuffle Mutation (PSM)</a>

<a href='#inversion'>06. Inversion Mutation or Reverse Sequence Mutation (RSM)</a>

<a href='#inorder'>07. Inorder Mutation</a>

<a href='#central-inversion'>08. Central Inversion Mutation (CIM)</a>

<a href='#throas'>09. Throas Mutation</a>

<a href='#thrors'>10. Thrors Mutation</a>

<a href='#distance-based'>11. Distance-Based Mutation (DMO)</a>

<a href='#displacement'>12. Displacement Mutation</a>

<a href='#insertion'>13. Insertion Mutation</a>

<a href='#displaced-inversion'>14. Displaced Inversion Mutation</a>

<a href='#creep'>15. Creep Mutation</a>

<a href='#uniform-random'>16. Uniform Random Mutation</a>


## <a id='chrom-def'>Chromosome Definition</a>

First, we need to implement the chromosome class, a function to calculate the fitness of individuals (chromosomes) in the population, and some helper functions.

In [2]:
import numpy as np

class Chromosome():
    """
    Description of class `Chromosome`:
    This class represents a simple chromosome. In the method describe, a simple description
    of the chromosome is provided, when it is called. 
    """
    def __init__(self, genes, id_=None, fitness=-1):
        self.id_ = id_
        self.genes = genes
        self.fitness = fitness       
       
    def describe(self): 
        """
        Prints the ID, fitness, and genes
        """
        print(f"ID=#{self.id_}, Fitness={self.fitness}, \nGenes=\n{self.genes}")
 
    def get_chrom_length(self): 
        """
        Returns the length of `self.genes`
        """
        return len(self.genes)

def find_chrom_type(chrom):
    """
    This function, takes a chromosome and returns its type (binary, integer, 
    or a floating-point chromosome).
    chrom: The chromoseme which its type is calculated and returned.
    """ 
    # if a floating-point individual
    if np.issubdtype(chrom.dtype, np.floating):
        return "float"
    # if a binary individual
    elif np.array_equal(chrom, chrom.astype(bool)):
        return "binary"
    # if an integer individual
    elif np.issubdtype(chrom.dtype, np.integer):
        return "integer"
    
    return "binary"

# Note that calculating the fitness of a chromosome depends on the problem at hand,
# and what you see in this function is just for understanding the concept.
def fitness_function(chrom): 
    """
    This function, takes a chromosome and returns a value as its fitness.
    chrom: The chromoseme which its fitness is calculated and returned.
    """  
    fitness = 0
    chrom_type = find_chrom_type(chrom)
    # if we have a binary chromosome (count the number of 1s)
    if chrom_type == "binary":
        return np.count_nonzero(chrom == 1)
    # if we have an integer chromosome (count values between 0 and 4)
    elif chrom_type == "integer":
        return np.count_nonzero((chrom >= 0) & (chrom <= 4))
    # if we have a floating-point chromosome (count values between 0 and 0.5) 
    elif chrom_type == "float":
        return np.count_nonzero((chrom >= 0) & (chrom <= 0.5))
    
    return -1


def sorted_random_numbers(point_nums, interval):
    """
    Returns several sorted random integers in an interval.
    point_nums: number of points to be selected.
    interval: the interval which the numbers are selected.
    """    
    points = []
    for _ in range(point_nums):
        num = np.random.randint(1, abs(interval))
        while num in points:
            num = np.random.randint(1, abs(interval))
        points.append(num)
    return sorted(points)


def integer_chrom_generator(n):
    """
    Produces chromosomes with integer values for each gene. (integer alleles)
    n: The length of the chromosome to be produced 
    """  
    a = []
    for _ in range(n):
        rand = np.random.randint(0, n)
        while rand in a:
            rand = np.random.randint(0, n)
        else:
            a.append(rand)
    return np.array(a)


def binary_chrom_generator(n, zero_prob, one_prob): 
    """
    Produces chromosomes with binary values for each gene. (binary alleles)
    n: The length of the chromosome to be produced 
    zero_prob: The probability for the allele to be zero
    one_prob: The probability for the allele to be one
    """   
    return np.random.choice([0, 1], size=n, p=[zero_prob, one_prob])



## <a id='bit-flip'>Bit Flip Mutation</a>

In bit-flip mutation, one or several positions are randomly selected, and their values are flipped. In other words, in the selected positions, if the value of the gene is 1, it will become 0; and if the value of the gene is 0, it will become 1.

<figure>
    <center> <img src="./pics/Mutation/bit_flip_mutation.png"  alt='missing' width="450"  ><center/>
<figure/>

In [18]:
import numpy as np
from copy import deepcopy

def bit_flip_mutation(parent_one, pm):
    """
    This function takes one chromosome, and performs Bit flip mutation on it. 
    parent_one: The parent
    pm: The probability of mutation
    """ 
    
    print("\nParent")
    print("=================================================")
    Chromosome.describe(parent_one)
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   

    if np.random.rand() < pm:  # if pm is greater than random number
        point_nums = np.random.randint(1, 4)
        points = sorted_random_numbers(point_nums, chrom_length)
        print("\npoints:", points)

        for i in range(point_nums): #performing XOR, to flip the bits in the chosen points
            parent_one.genes[points[i]] = parent_one.genes[points[i]] ^ 1
        child_one.genes = parent_one.genes

    else:  # if pm is less than random number then don't make any change
        child_one = deepcopy(parent_one)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    
    return child_one


chrom0 = np.array([1, 1, 1, 0, 1, 0, 0, 1, 0])
parent_one = Chromosome(genes= chrom0, id_=0, fitness = fitness_function(chrom0))   
MUTATION = bit_flip_mutation(parent_one, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(MUTATION)  #one child



Parent
ID=#0, Fitness=5, 
Genes=
[1 1 1 0 1 0 0 1 0]

points: [3, 7]

Children
ID=#0, Fitness=5, 
Genes=
[1 1 1 1 1 0 0 0 0]


## <a id='random-resetting'>Random Resetting Mutation</a>

In random resetting mutation, one or several positions are randomly selected, and a value is determined for each of those positions, from the allowed or valid domain. A domain could be the positive integers.

<figure>
    <center> <img src="./pics/Mutation/random_resetting_mutation.png"  alt='missing' width="450"  ><center/>
<figure/>

In [19]:
import numpy as np
from copy import deepcopy

def random_resetting_mutation(parent_one, pm):
    """
    This function takes one chromosome, and performs Random resetting mutation on it. 
    parent_one: The parent
    pm: The probability of mutation
    """ 
    parent_one = Chromosome(genes= np.array([1, 2, 3, 4, 5, 6, 7, 8, 9]),id_=0,fitness = 125.2)   
    print("\nParent")
    print("=================================================")
    Chromosome.describe(parent_one)
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   

    if np.random.rand() < pm:  # if pm is greater than random number
        point_nums = np.random.randint(1, 4)
        points = sorted_random_numbers(point_nums, chrom_length)
        print("\npoints:", points)

        for i in range(len(points)): 
            parent_one.genes[points[i]] = np.random.randint(1, 10) #one of the admissible values
        child_one.genes = parent_one.genes 

    else:  # if pm is less than random number then don't make any change
        child_one = deepcopy(parent_one)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    
    return child_one


chrom0 = np.array([1, 1, 1, 0, 1, 0, 0, 1, 0])
parent_one = Chromosome(genes= chrom0, id_=0, fitness = fitness_function(chrom0))   
MUTATION = random_resetting_mutation(parent_one, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(MUTATION)  #one child



Parent
ID=#0, Fitness=125.2, 
Genes=
[1 2 3 4 5 6 7 8 9]

points: [4, 7]

Children
ID=#0, Fitness=4, 
Genes=
[1 2 3 4 7 6 7 9 9]


## <a id='swap'>Swap Mutation</a>
In swap mutation, two genes are randomly selected, and the value of those two genes is swapped.

<figure>
    <center> <img src="./pics/Mutation/swap_mutation.png"  alt='missing' width="450"  ><center/>
<figure/>

In [5]:
import numpy as np
from copy import deepcopy

def swap_mutation(parent_one, pm):
    """
    This function takes one chromosome, and performs Swap mutation on it. 
    parent_one: The parent
    pm: The probability of mutation
    """ 

    print("\nParent")
    print("=================================================")
    Chromosome.describe(parent_one)
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   

    if np.random.rand() < pm:  # if pm is greater than random number
        points = sorted_random_numbers(2, chrom_length)
        print("\npoints:", points)    
        parent_one.genes[points[0]], parent_one.genes[points[1]] = parent_one.genes[points[1]], parent_one.genes[points[0]] 
        child_one.genes = parent_one.genes 

    else:  # if pm is less than random number then don't make any change
        child_one = deepcopy(parent_one)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    
    return child_one


chrom0 = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
parent_one = Chromosome(genes= chrom0, id_=0, fitness = fitness_function(chrom0))   
MUTATION = swap_mutation(parent_one, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(MUTATION)  #one child




Parent
ID=#0, Fitness=4, 
Genes=
[1 2 3 4 5 6 7 8 9]

points: [2, 6]

Children
ID=#0, Fitness=4, 
Genes=
[1 2 7 4 5 6 3 8 9]


## <a id='scramble'>Scramble Mutation or Partial Shuffle Mutation (PSM)</a>

In scramble mutation, a substring of the chromosome is selected and the value of its genes are shuffled.

<figure>
    <center> <img src="./pics/Mutation/scramble_mutation.png"  alt='missing' width="450"  ><center/>
<figure/>

In [20]:
import numpy as np
from copy import deepcopy

def scramble_mutation(parent_one, pm):
    """
    This function takes one chromosome, and performs Scramble mutation on it. 
    parent_one: The parent
    pm: The probability of mutation
    """ 

    print("\nParent")
    print("=================================================")
    Chromosome.describe(parent_one)
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   

    if np.random.rand() < pm:  # if pm is greater than random number
        points = sorted_random_numbers(2, chrom_length)   
        permute = []
        child_one.genes = parent_one.genes
        for i in range(points[0], points[1]):
            permute.append(parent_one.genes[i])
        np.random.shuffle(permute)
        k = 0
        for i in range(points[0], points[1]):
            child_one.genes[i] = permute[k]
            k+=1      
        
    else:  # if pm is less than random number then don't make any change
        child_one = deepcopy(parent_one)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    
    return child_one


chrom0 = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
parent_one = Chromosome(genes= chrom0, id_=0, fitness = fitness_function(chrom0))   
MUTATION = scramble_mutation(parent_one, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(MUTATION)  #one child



Parent
ID=#0, Fitness=4, 
Genes=
[1 2 3 4 5 6 7 8 9]

Children
ID=#0, Fitness=4, 
Genes=
[1 4 2 7 5 3 6 8 9]


## <a id='inversion'>Inversion Mutation or Reverse Sequence Mutation (RSM)</a>

In inversion mutation, a substring of the chromosome is selected and is inversed and replaced.

<figure>
    <center> <img src="./pics/Mutation/inversion_mutation.png"  alt='missing' width="450"  ><center/>
<figure/>

In [22]:
import numpy as np
from copy import deepcopy

def inversion_mutation(parent_one, pm):
    """
    This function takes one chromosome, and performs Inversion mutation on it. 
    parent_one: The parent
    pm: The probability of mutation
    """ 

    print("\nParent")
    print("=================================================")
    Chromosome.describe(parent_one)
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   

    if np.random.rand() < pm:  # if pm is greater than random number
        points = sorted_random_numbers(2, chrom_length)  
        reverse_list = []
        child_one.genes = parent_one.genes
        for i in range(points[0], points[1]):
            reverse_list.append(parent_one.genes[i])
        reverse_list.reverse()
        # print(reverse_list)
        k = 0
        for i in range(points[0], points[1]):
            child_one.genes[i] = reverse_list[k]
            k+=1      
         
    else:  # if pm is less than random number then don't make any change
        child_one = deepcopy(parent_one)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    
    return child_one


chrom0 = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
parent_one = Chromosome(genes= chrom0, id_=0, fitness = fitness_function(chrom0))   
MUTATION = inversion_mutation(parent_one, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(MUTATION)  #one child




Parent
ID=#0, Fitness=4, 
Genes=
[1 2 3 4 5 6 7 8 9]

Children
ID=#0, Fitness=4, 
Genes=
[1 8 7 6 5 4 3 2 9]


## <a id='inorder'>Inorder Mutation</a>

In Inorder mutation, several random positions are selected, and for each position, a uniform random real number `w` is produced. If `w < 0.5` for that position, the value of the gene in that position is flipped. Otherwise, the value of the gene will remain unchanged.

<figure>
    <center> <img src="./pics/Mutation/inorder_mutation.png"  alt='missing' width="450"  ><center/>
<figure/>

In [8]:
import numpy as np
from copy import deepcopy

def inorder_mutation(parent_one, pm):
    """
    This function takes one chromosome, and performs Inorder mutation on it. 
    parent_one: The parent
    pm: The probability of mutation
    """ 
    
    print("\nParent")
    print("=================================================")
    Chromosome.describe(parent_one)
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   

    if np.random.rand() < pm:  # if pm is greater than random number
        point_nums = np.random.randint(1, 5) #selecting one to four elements 
        points = sorted_random_numbers(point_nums, chrom_length)
        child_one.genes = parent_one.genes
        print("\npoints:", points)    
        for i in range(point_nums):
            w = np.random.uniform(low=0, high=1)
            if w < 0.5:
                child_one.genes[points[i]] = child_one.genes[points[i]] ^ 1 #flipping the bit

    else:  # if pm is less than random number then don't make any change
        child_one = deepcopy(parent_one)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    
    return child_one



chrom0 = np.array([0, 1, 0, 1, 1, 0, 1, 1, 0])
parent_one = Chromosome(genes= chrom0, id_=0, fitness = fitness_function(chrom0))   
MUTATION = inorder_mutation(parent_one, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(MUTATION)  #one child




Parent
ID=#0, Fitness=5, 
Genes=
[0 1 0 1 1 0 1 1 0]

points: [2, 5, 6, 7]

Children
ID=#0, Fitness=5, 
Genes=
[0 1 1 1 1 0 0 1 0]


## <a id='center-inversion'>Center Inversion Mutation (CIM)</a>

In center inversion mutation, we divide the chromosome into two parts; then we reverse each part and transfer the result to the offspring.

<figure>
    <center> <img src="./pics/Mutation/center_inversion_mutation.png"  alt='missing' width="450"  ><center/>
<figure/>

In [9]:
import numpy as np
from copy import deepcopy

def center_inversion_mutation(parent_one, pm):
    """
    This function takes one chromosome, and performs Center inversion mutation on it. 
    Length of the chromosome in this mutation should be more than 4.
    parent_one: The parent
    pm: The probability of mutation
    """ 

    print("\nParent")
    print("=================================================")
    Chromosome.describe(parent_one)
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   

    if np.random.rand() < pm:  # if pm is greater than random number
        reverse_list1 = []
        reverse_list2 = []
        point = np.random.randint(2,chrom_length-1) #break point  
        child_one.genes = parent_one.genes
        for i in range(chrom_length):
            if i < point:
                reverse_list1.append(parent_one.genes[i])
            else:
                reverse_list2.append(parent_one.genes[i])
        reverse_list1.reverse()
        reverse_list2.reverse()

        for i in range(point): #Filling the first half
            child_one.genes[i] = reverse_list1[i]
        k = 0
        for i in range(point, chrom_length): #Filling the first half
            child_one.genes[i] = reverse_list2[k]
            k+=1
                
    else:  # if pm is less than random number then don't make any change
        child_one = deepcopy(parent_one)
    
    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)

    return child_one



chrom0 = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
parent_one = Chromosome(genes= chrom0, id_=0, fitness = fitness_function(chrom0))   
MUTATION = center_inversion_mutation(parent_one, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(MUTATION)  #one child




Parent
ID=#0, Fitness=4, 
Genes=
[1 2 3 4 5 6 7 8 9]

Children
ID=#0, Fitness=4, 
Genes=
[4 3 2 1 9 8 7 6 5]


## <a id='throas'>Throas Mutation</a>
In throas mutation, we select three positions in a sequence and do the following. <br>
> (1) Place the last element in the first position in the child.<br>
> (2) Place the second element in the last position in the child.<br>
> (3) Place the first element in the second position in the child.

All the other elements are directly transferred from parent-one to child-one.

<figure>
    <center> <img src="./pics/Mutation/throas_mutation.png"  alt='missing' width="450"  ><center/>
<figure/>

In [10]:
import numpy as np
from copy import deepcopy

def throas_mutation(parent_one, pm):
    """
    This function takes one chromosome, and performs Throas mutation on it. 
    Last ==> First
    Second ==> Last
    First ==> Second 
    parent_one: The parent
    pm: The probability of mutation
    """ 

    print("\nParent")
    print("=================================================")
    Chromosome.describe(parent_one)
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   

    if np.random.rand() < pm:  # if pm is greater than random number
        interval = []
        point1 = np.random.randint(0,chrom_length-3)
        point2 = point1 + 3
        child_one.genes = parent_one.genes
        for i in range(point1, point2):
            interval.append(child_one.genes[i])
        # print(interval)
        interval_copy = interval.copy()
        interval[0] = interval_copy[-1] 
        interval[-1] = interval_copy[1] 
        interval[1] = interval_copy[0] 
        # print(interval)
        k = 0
        for i in range(point1, point2):
            child_one.genes[i] = interval[k]
            k+=1

    else:  # if pm is less than random number then don't make any change
        child_one = deepcopy(parent_one)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    
    return child_one


chrom0 = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
parent_one = Chromosome(genes= chrom0, id_=0, fitness = fitness_function(chrom0))   
MUTATION = throas_mutation(parent_one, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(MUTATION)  #one child




Parent
ID=#0, Fitness=4, 
Genes=
[1 2 3 4 5 6 7 8 9]

Children
ID=#0, Fitness=4, 
Genes=
[1 2 5 3 4 6 7 8 9]


## <a id='thrors'>Thrors Mutation</a>

In thrors mutation, we select three positions that are not necessarily in a sequence and do the following.<br>
> (1) Place the last element in the first position in the child. <br>
> (2) Place the second element in the last position in the child. <br>
> (3) Place the first element in the second position in the child.

All the other elements are directly transferred from parent-one to child-one.

<figure>
    <center> <img src="./pics/Mutation/thrors_mutation.png"  alt='missing' width="450"  ><center/>
<figure/>

In [11]:
import numpy as np
from copy import deepcopy

def thrors_mutation(parent_one, pm):
    """
    This function takes one chromosome, and performs Thrors mutation on it. 
    Last ==> First
    Second ==> Last
    First ==> Second 
    In compare to Throas mutation, in Thrors mutation, the chosen elements are not necessarily 
    successive. 
    parent_one: The parent
    pm: The probability of mutation
    """ 

    print("\nParent")
    print("=================================================")
    Chromosome.describe(parent_one)
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   

    if np.random.rand() < pm:  # if pm is greater than random number
        elements = []
        points = sorted_random_numbers(3, chrom_length)
        # print(points)    
        child_one.genes = parent_one.genes

        for i in range(len(points)): #selecting the elements
            elements.append(parent_one.genes[points[i]]) 
        # print(elements)
        elements_copy = elements.copy()
        elements[0] = elements_copy[-1] 
        elements[-1] = elements_copy[1] 
        elements[1] = elements_copy[0] 
        # print(elements)
        k = 0
        for i in range(chrom_length):
            if i in points:
                child_one.genes[i] = elements[k]
                k+=1
    else:  # if pm is less than random number then don't make any change
        child_one = deepcopy(parent_one)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    
    return child_one



chrom0 = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
parent_one = Chromosome(genes= chrom0, id_=0, fitness = fitness_function(chrom0))   
MUTATION = thrors_mutation(parent_one, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(MUTATION)  #one child




Parent
ID=#0, Fitness=4, 
Genes=
[1 2 3 4 5 6 7 8 9]

Children
ID=#0, Fitness=4, 
Genes=
[1 8 2 4 5 6 7 3 9]


## <a id='distance-based'>Distance-Based Mutation (DMO)</a>

In distance-based mutation, we select a random position and find the elements at a certain distance around the selected position. After finding the selected elements, we shuffle the whole interval.

To calculate the selected elements, we define two variables: the variable `g` which indicates the randomly selected position, and the variable `distance` to indicate which elements on either side of the randomly selected element will be selected.

<figure>
    <center> <img src="./pics/Mutation/distance_based_mutation.png"  alt='missing' width="450"  ><center/>
<figure/>

In [12]:
import numpy as np
from copy import deepcopy

def distance_based_mutation(parent_one, pm):
    """
    This function takes one chromosome, and performs Distance based mutation on it.
    In this method, g is the position for the chosen gene. d is the distance from that position. 
    parent_one: The parent
    pm: The probability of mutation
    """ 

    print("\nParent")
    print("=================================================")
    Chromosome.describe(parent_one)
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   

    if np.random.rand() < pm:  # if pm is greater than random number
        interval = []
        d = np.random.randint(1, 4)
        g = np.random.randint(d,chrom_length-d)
        print("\ng = {}, distance = {}".format(g, d))
        child_one.genes = parent_one.genes
        for i in range(g - d, g + d + 1): #element in position g and d elements in each direction
            interval.append(parent_one.genes[i]) 
        np.random.shuffle(interval)
        k = 0
        for i in range(g - d, g + d + 1): #Replacing elements
            child_one.genes[i] = interval[k]
            k+=1


    else:  # if pm is less than random number then don't make any change
        child_one = deepcopy(parent_one)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    
    return child_one


chrom0 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
parent_one = Chromosome(genes= chrom0, id_=0, fitness = fitness_function(chrom0))   
MUTATION = distance_based_mutation(parent_one, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(MUTATION)  #one child




Parent
ID=#0, Fitness=5, 
Genes=
[0 1 2 3 4 5 6 7 8 9]

g = 4, distance = 3

Children
ID=#0, Fitness=5, 
Genes=
[0 5 2 1 7 3 4 6 8 9]


## <a id='displacement'>Displacement Mutation</a>

In displacement mutation, we select a random substring from the chromosome, and paste it in another position, without changing the sequence of other elements.

<figure>
    <center> <img src="./pics/Mutation/displacement_mutation.png"  alt='missing' width="450"  ><center/>
<figure/>

In [13]:
import numpy as np
from copy import deepcopy

def displacement_mutation(parent_one, pm):
    """
    This function takes one chromosome, and performs Displacement mutation on it. 
    parent_one: The parent
    pm: The probability of mutation
    """ 

    print("\nParent")
    print("=================================================")
    Chromosome.describe(parent_one)
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   

    if np.random.rand() < pm:  # if pm is greater than random number
        interval = []
        points = sorted_random_numbers(2, chrom_length)
        # print(points)    
        duplicate = parent_one.genes.tolist().copy() 
        for i in range(points[0], points[1]):
            interval.append(parent_one.genes[i])
        # print(interval)
        for i in range(chrom_length):
            if parent_one.genes[i] in interval:
                duplicate.remove(parent_one.genes[i])
   
        # print(duplicate)
        lists = []
        for i in range(len(duplicate) + 1):
            a = []
            lists.append(a)
        position = np.random.randint(0, len(duplicate) + 1) #the position of the paste
        # print(position)
        for i in range(len(interval)):
            lists[position].append(interval[i])
        k = 0
        for i in range(len(lists)):
            if not lists[i]: #if lists[i] is empty
                lists[i].append(duplicate[k])
                k+=1
        # print(lists)
        flat_list = [item for sublist in lists for item in sublist]
        child_one.genes = np.array(flat_list)

    else:  # if pm is less than random number then don't make any change
        child_one = deepcopy(parent_one)
    
    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)

    return child_one


chrom0 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
parent_one = Chromosome(genes= chrom0, id_=0, fitness = fitness_function(chrom0))   
MUTATION = displacement_mutation(parent_one, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(MUTATION)  #one child




Parent
ID=#0, Fitness=5, 
Genes=
[0 1 2 3 4 5 6 7 8 9]

Children
ID=#0, Fitness=5, 
Genes=
[0 7 8 9 1 2 3 4 5 6]


## <a id='insertion'>Insertion Mutation</a>

In insertion mutation, we select a random gene in the chromosome and we paste that gene in another position, without changing the sequence of other elements. The main difference between insertion mutation and displacement mutation is that in insertion mutation, we change the position of only one element.

<figure>
    <center> <img src="./pics/Mutation/insertion_mutation.png"  alt='missing' width="450"  ><center/>
<figure/>

In [14]:
import numpy as np
from copy import deepcopy

def insertion_mutation(parent_one, pm):
    """
    This function takes one chromosome, and performs Insertion mutation on it. 
    parent_one: The parent
    pm: The probability of mutation
    """ 

    print("\nParent")
    print("=================================================")
    Chromosome.describe(parent_one)
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   

    if np.random.rand() < pm:  # if pm is greater than random number
        point = np.random.randint(0, chrom_length) #picking one gene
        # print(point)    
        duplicate = parent_one.genes.tolist().copy()         
        duplicate.remove(parent_one.genes[point]) 
        # print(duplicate)
        
        lists = []
        for i in range(len(duplicate) + 1):
            a = []
            lists.append(a)
        position = np.random.randint(0, len(duplicate) + 1) #the position of the paste
        # print(position)
        lists[position].append(parent_one.genes[point])
        k = 0
        for i in range(len(lists)):
            if not lists[i]: #if lists[i] is empty
                lists[i].append(duplicate[k])
                k+=1
        # print(lists)
        flat_list = [item for sublist in lists for item in sublist]
        child_one.genes = np.array(flat_list)

    else:  # if pm is less than random number then don't make any change
        child_one = deepcopy(parent_one)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    
    return child_one


chrom0 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
parent_one = Chromosome(genes= chrom0, id_=0, fitness = fitness_function(chrom0))   
MUTATION = insertion_mutation(parent_one, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(MUTATION)  #one child



Parent
ID=#0, Fitness=5, 
Genes=
[0 1 2 3 4 5 6 7 8 9]

Children
ID=#0, Fitness=5, 
Genes=
[0 2 3 4 1 5 6 7 8 9]


## <a id='displaced-inversion'>Displaced Inversion Mutation</a>

In displaced inversion mutation, we select a random substring from the chromosome, reverse that substring, and paste it in another position, without changing the sequence of other elements.

<figure>
    <center> <img src="./pics/Mutation/displaced_inversion_mutation.png"  alt='missing' width="450"  ><center/>
<figure/>

In [15]:
import numpy as np
from copy import deepcopy

def displaced_inversion_mutation(parent_one, pm):
    """
    This function takes one chromosome, and performs Displaced inversion mutation on it. 
    parent_one: The parent
    pm: The probability of mutation
    """ 

    print("\nParent")
    print("=================================================")
    Chromosome.describe(parent_one)
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   

    if np.random.rand() < pm:  # if pm is greater than random number
        interval = []
        points = sorted_random_numbers(2, chrom_length)
        # print(points)    
        duplicate = parent_one.genes.tolist().copy() 
        for i in range(points[0], points[1]):
            interval.append(parent_one.genes[i])
        # print(interval)
        for i in range(chrom_length):
            if parent_one.genes[i] in interval:
                duplicate.remove(parent_one.genes[i])
   
        # print(duplicate)
        lists = []
        for i in range(len(duplicate) + 1):
            a = []
            lists.append(a)
        position = np.random.randint(0, len(duplicate) + 1) #the position of the paste
        # print(position)
        interval.reverse()
        for i in range(len(interval)):
            lists[position].append(interval[i])

        k = 0
        for i in range(len(lists)):
            if not lists[i]: #if lists[i] is empty
                lists[i].append(duplicate[k])
                k+=1
        # print(lists)
        flat_list = [item for sublist in lists for item in sublist]
        child_one.genes = np.array(flat_list)

    else:  # if pm is less than random number then don't make any change
        child_one = deepcopy(parent_one)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    
    return child_one


chrom0 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
parent_one = Chromosome(genes= chrom0, id_=0, fitness = fitness_function(chrom0))   
MUTATION = displaced_inversion_mutation(parent_one, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(MUTATION)  #one child



Parent
ID=#0, Fitness=5, 
Genes=
[0 1 2 3 4 5 6 7 8 9]

Children
ID=#0, Fitness=5, 
Genes=
[0 1 2 5 6 7 4 3 8 9]


## <a id='creep'>Creep Mutation</a>

**Creep mutation** is a type of mutation used in **real-value encoding schemes** in genetic algorithms. It involves the addition of a small random vector to the elements of a chromosome or an element-wise approach involving a probability test on each element to determine whether to perform the mutation. 

In the following implementation, we choose a random position in the parent chromosome and replace its value with a random value chosen from the valid domain (in our case, in `[0, 1]`). 


<figure>
    <center> <img src="./pics/Mutation/creep_mutation.png"  alt='missing' width="450"  ><center/>
<figure/>

In [16]:
import numpy as np
from copy import deepcopy

def creep_mutation(parent_one, pm):
    """
    This function takes one chromosome, and performs Creep mutation on it. 
    parent_one: The parent
    pm: The probability of mutation
    """ 
    print("\nParent")
    print("=================================================")
    Chromosome.describe(parent_one)
    chrom_length = Chromosome.get_chrom_length(parent_one)
    child_one = Chromosome(genes= np.array([0.0]*chrom_length),id_=0,fitness = 125.2)   

    if np.random.rand() < pm:  # if pm is greater than random number
        point = np.random.randint(0, chrom_length)
        print("\nChosen point:", point)    
        child_one.genes = parent_one.genes 
        child_one.genes[point] = round(np.random.uniform(low=0.1, high=0.95), 3)

    else:  # if pm is less than random number then don't make any change
        child_one = deepcopy(parent_one)
    
    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    
    return child_one



chrom0 = np.array([0.121, 0.152, 0.231, 0.143, 0.732, 0.315, 0.434, 0.633])
parent_one = Chromosome(genes= chrom0, id_=0, fitness = fitness_function(chrom0))   
MUTATION = creep_mutation(parent_one, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(MUTATION)  #one child





Parent
ID=#0, Fitness=6, 
Genes=
[0.121 0.152 0.231 0.143 0.732 0.315 0.434 0.633]

Chosen point: 5

Children
ID=#0, Fitness=5, 
Genes=
[0.121 0.152 0.231 0.143 0.732 0.703 0.434 0.633]


## <a id='uniform-random'>Uniform Random Mutation</a>

In **Uniform Random Mutation**, a new individual is created by randomly generating a value for each gene in the chromosome. The new individual is then subjected to uniform crossover with the original individual. 

This mutation operator is used to introduce new genetic material into the population and to maintain genetic diversity. 



<figure>
    <center> <img src="./pics/Mutation/uniform_random.png"  alt='missing' width="450"  ><center/>
<figure/>

In [17]:
import numpy as np
from copy import deepcopy

def claculate_mask(chrom_size, px):
    """
    This method, performs the mask calculation for the Uniform crossover.
    chrom_size: The mask has the same size of the chromosome.
    px: A control parameter for the number of ones in the mask vector.
    """  
    mask = [0]*chrom_size
    for i in range(chrom_size):
        prob = np.random.rand()
        if prob <= px:
            mask[i]=1
    return mask


def uniform_random_mutation(parent_one, pm):
    """
    parent_one: The parent
    pm: The probability of mutation
    """ 

    print("\nParent")
    print("=================================================")
    Chromosome.describe(parent_one)
    chrom_length = Chromosome.get_chrom_length(parent_one)
    chrom_type = find_chrom_type(parent_one.genes)

    
    if chrom_type == "float":
        random_genes = np.round(np.random.uniform(0.0, 1.0, chrom_length), 3)
        parent_two = Chromosome(genes= random_genes, id_=0, fitness = fitness_function(random_genes))
        child_one = Chromosome(genes= np.array([0.0]*chrom_length),id_=0,fitness = 125.2)   
    
    elif chrom_type == "integer":
        random_genes = integer_chrom_generator(chrom_length)
        parent_two = Chromosome(genes= random_genes, id_=0, fitness = fitness_function(random_genes))
        child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   
    
    else: # binary individual
        random_genes = np.array(binary_chrom_generator(chrom_length, 0.5, 0.5))
        parent_two = Chromosome(genes= random_genes, id_=0, fitness = fitness_function(random_genes))
        child_one = Chromosome(genes= np.array([-1]*chrom_length),id_=0,fitness = 125.2)   

    # parent_two is a randomly generated chromosome (not in the population)
    Chromosome.describe(parent_two)

    if np.random.rand() < pm:  # if pm is greater than random number
        mask = claculate_mask(chrom_length, np.random.uniform(low=0.2, high=0.85))
        print("\nMask vector is:", mask)
        for i in range(chrom_length):
            if mask[i]==1:
                child_one.genes[i] = parent_one.genes[i]
            else:
                child_one.genes[i] = parent_two.genes[i]         

    else:  # if pm is less than random number then don't make any change
        child_one = deepcopy(parent_one)

    # updating fitnesses for children
    child_one.fitness = fitness_function(child_one.genes)
    
    return child_one



chrom0 = np.array([0.121, 0.152, 0.231, 0.143, 0.732, 0.315, 0.434, 0.633])
chrom1 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
chrom2 = np.array([0, 1, 0, 0, 1, 0, 0, 1, 1, 0])
parent_one = Chromosome(genes= chrom0, id_=0, fitness = fitness_function(chrom0))   
parent_two = Chromosome(genes= chrom1, id_=1, fitness = fitness_function(chrom1))   
parent_three = Chromosome(genes= chrom2, id_=2, fitness = fitness_function(chrom2))   

# You can try this mutation with any kind of representation (e.g., float, binary, integer, etc.)
MUTATION = uniform_random_mutation(parent_one, 1)
# MUTATION = uniform_random_mutation(parent_two, 1)
# MUTATION = uniform_random_mutation(parent_three, 1)

if __name__ == '__main__':
    print("\nChildren")
    print("=================================================")
    Chromosome.describe(MUTATION)  #one child





Parent
ID=#0, Fitness=6, 
Genes=
[0.121 0.152 0.231 0.143 0.732 0.315 0.434 0.633]
ID=#0, Fitness=1, 
Genes=
[0.904 0.903 0.576 0.759 0.341 0.612 0.721 0.934]

Mask vector is: [0, 1, 1, 1, 0, 1, 1, 1]

Children
ID=#0, Fitness=6, 
Genes=
[0.904 0.152 0.231 0.143 0.341 0.315 0.434 0.633]


## References 

1. Engelbrecht, A. P. (2007). Computational intelligence: An introduction (2nd ed.). John Wiley & Sons.

2. Bing, M. (2023). Recombination in genetic algorithms [Graphic art]. Image Creator from Microsoft Designer.

3. Optimization and Parameter Estimation, Genetic Algorithms. (n.d.). In SpringerLink. Retrieved December 27, 2023, from https://link.springer.com/referenceworkentry/10.1007/978-1-4419-9863-7_291

4. Zhang, Y., & Wang, X. (2021). A Method Enabling Comprehensive Isolation of Arabidopsis Root Cell Types for Transcriptional Analysis. Frontiers in Plant Science, 12, 646404. https://doi.org/10.3389/fpls.2021.646404

5. Creepy Mutant. (n.d.). In The Forest Wiki. Retrieved December 27, 2023, from https://theforest.fandom.com/wiki/Creepy_Mutant

6. Koza, J. R. (1992). Exact GP schema theory for headless chicken crossover and subtree mutation. In Proceedings of the 4th International Conference on Genetic Algorithms (pp. 78-85). IEEE. https://doi.org/10.1109/ICGA.1991.576974

7. Gorman, J. (2015, March 11). Why Transylvanian Chickens Have Naked Necks. National Geographic. Retrieved December 27, 2023, from https://www.nationalgeographic.com/animals/article/110315-transylvanian-naked-neck-chicken-churkeys-turkens-science

8. Kummer, C. (2014, August 27). Here's Why a Chicken Can Live Without Its Head. Modern Farmer. Retrieved December 27, 2023, from https://modernfarmer.com/2014/08/heres-chicken-can-live-without-head/

9. Li, Y., Wu, F., & Zhang, Y. (2020). The genetic basis and robustness of naked neck mutation in chicken: A comprehensive analysis. Tropical Animal Health and Production, 52(6), 3049-3056. https://doi.org/10.1007/s11250-020-02505-1

10. IEEE Xplore Digital Library. (n.d.). Retrieved December 27, 2023, from https://ieeexplore.ieee.org/servlet/opac?punumber=7440.

